<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/Railway_Accident_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install nltk
!pip install pytorch-crf

In [3]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

from transformers import BertTokenizer

import json
import glob
import re
import os
import ast
import html

from google.colab import files

In [6]:
#디바이스 세팅
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# google api key
key = files.upload()

Saving railway-tagtog-d24e6901ddb0.json to railway-tagtog-d24e6901ddb0.json


#Sample Annotation : JSON to csv
by TAGTOG

In [8]:
import os
import requests

#로그인 위치
url = "https://tagtog.com/-login"

#다운로드 위치
file_url = 'https://www.tagtog.com/slayerzeroa/Railway_BERT/-downloads/dataset-as-anndoc' 
o_file = 'abc.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : 'slayerzeroa@gmail.com',
    'password' : 'gyysxw5rU3NzrYX',
}

#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [9]:
#압축파일 풀기
import zipfile
import shutil
folder_path = "./tagtog_relation_extraction/"

zip_ = zipfile.ZipFile("abc.zip")
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

In [10]:
import json
import glob
import re
import os
#폴더 경로
folder_name = "./tagtog_relation_extraction/Railway_BERT/"

#context list
context_name_list = os.listdir(folder_name + "plain.html/pool")
print(context_name_list)

#relation 폴더 경로
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool")

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]

print(contexts_folders_paths)

#anntation_lenged 정보
annotation_legend = folder_name + "annotations-legend.json"
with open(annotation_legend,"r") as f:
    annotation_legend = json.load(f)

print(annotation_legend)

['a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html']
['./tagtog_relation_extraction/Railway_BERT/plain.html/pool/a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html']
{'e_23': 'SOLUTION', 'r_19': 'mz4f90xj280(e_6|e_4)', 'r_26': '5zjoi2gg5ps(e_6|e_23)', 'e_22': 'ACCIDENT', 'r_25': '6oyjvcblqfe(e_22|e_23)', 'e_6': 'IMPLIED', 'r_27': 'f19f8drqjkn(e_4|e_23)', 'e_4': 'CAUSE', 'r_24': 'yz4ctet7cda(e_4|e_22)'}


In [ ]:
#relation dictionary 파일로부터 subject와 object의 entity 정보를 추출해주는 함수
def get_needed_relation_data(tmp_relation):
    subject_token = re.findall("\(+(.+)+\)",annotation_legend[tmp_relation["relations"][0]['classId']])[0].split("|")[0]
    if subject_token == tmp_relation['entities'][0]['classId']:
        cause, implied = tmp_relation['entities']
    else:
        implied, cause  = tmp_relation['entities']
    
    # get preprocessed entities
    def _get_entity(entity):
        outputs = entity['offsets'][0]
        outputs['type'] = annotation_legend[entity['classId']].split("-")[0].lower()
        return outputs
    
    output_cause = _get_entity(cause)
    output_implied = _get_entity(implied)
    return output_cause, output_implied

In [ ]:
#relation dictionary 파일로부터 title(relation)을 추출해주는 함수
def get_label(relation_json):
    label_tag = relation_json['relations'][0]['classId'] #r_6
    try:
        sub_type, label = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-"), label_tag
        return f"{sub_type}:{label}"
    except:
        sub_type, = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-")
        return f"{sub_type}:no_relation"

In [ ]:
#html 파일로부터 text만 추출해주는 함수
def get_context_from_html(html_file):
    html_file = re.sub('(<([^>]+)>)', '', html_file)
    return html_file

In [ ]:
#entity 정보가 포함된 sentence를 생성해주는 함수
def get_sentence_with_entites(subject_entity, object_entity, sentence):
    if subject_entity['start'] < object_entity['start']:
        entity1,entity2 = subject_entity, object_entity
    else:
        entity1,entity2 = object_entity, subject_entity
    
    #entity 시작 위치 및 길이 
    ett1_stt, ett1_len = entity1['start'], len(entity1['text'])
    ett2_stt, ett2_len = entity2['start'], len(entity2['text'])
    
    #문장 분리
    bf, ett1, mid, ett2, af = sentence[:ett1_stt], \
                            sentence[ett1_stt:ett1_stt+ett1_len], \
                            sentence[ett1_stt+ett1_len:ett2_stt], \
                            sentence[ett2_stt:ett2_stt+ett2_len], \
                            sentence[ett2_stt+ett2_len:]

    
    if subject_entity['start'] < object_entity['start']:
        ett1,ett2 = f"<sbj:{ett1}>", f"<obj:{ett2}>"
    else:
        ett1,ett2 = f"<obj:{ett1}>", f"<sbj:{ett2}>"
    
    return "".join([bf, ett1, mid, ett2, af])

In [ ]:
#dataframe column
# id : context title(e.g : 카카오게임, 11번가 등)
# sentence w/o entity
# sentence w entity
# subject_entity
# object_entity
# class

id_list = []
sentence_list = []
sentence_with_entities_list = []
subject_entity_list = []
object_entity_list = []
relation_list = []
# tagtog 데이터를 CSV 형태로 변경
for context_name, relation_folder, contexts_folder in zip(context_name_list, relation_folder_paths, contexts_folders_paths):
    # relation files와 context files 리스트 출력
    file_ids = [file_name for file_name in os.listdir(relation_folder)]
    file_nums = [ids.split("-")[1] for ids in file_ids]
    relation_files = [relation_folder + "/"+ file_id for file_id in file_ids]
    context_files = [contexts_folder for file_id in file_ids]

    #json으로 된 relation data와 html로 된 context 데이터 읽기
    for relation_file, context_file, file_num in zip(relation_files,context_files, file_nums):
        #subject, object 정보 추출
        with open(relation_file, "r") as f:
            relation_json = json.load(f)
            tmp_subject, tmp_object = get_needed_relation_data(relation_json) #subject, object
            tmp_label = get_label(relation_json)
        try:
            tmp_subject, tmp_object = get_needed_relation_data(relation_json) #subject, object
            tmp_label = get_label(relation_json)
        except:
            continue
        #sentence, sentence with entities 정보 추출
        with open(context_file, "r") as f:
            context_json = f.read()
        tmp_sentence = get_context_from_html(context_json)
        tmp_sentence_w_entities = get_sentence_with_entites(tmp_subject,tmp_object,tmp_sentence)
        
        #각 list에 데이터 저장
        id_list.append(f"{context_name}_{file_num}")
        sentence_list.append(tmp_sentence)
        sentence_with_entities_list.append(tmp_sentence_w_entities)
        subject_entity_list.append(tmp_subject)
        object_entity_list.append(tmp_object)
        relation_list.append(tmp_label.lower())

In [ ]:
print(subject_entity_list)

[{'start': 3704, 'text': '화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것', 'type': 'implied'}]


In [11]:
# 참고 자료 : 

# !pip install gspread
# !pip install oauth2client

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './railway-tagtog-d24e6901ddb0.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
## gc.create(spreadsheet_name) # 스프레드시트 생성


#시트 선택하기
sheet_name = "annotation"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

In [ ]:
# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)
column_list = ["id","sentence","sentence_with_entity","subject_entity","object_entity","class"]
data = data[column_list]
data.head()

sen_list =  list(data.sentence_with_entity.values)
len(sen_list)

worksheet.resize(len(id_list)+1,10)
list_range = f"a2:f{len(id_list)+1}"
cell_list = worksheet.range(list_range)

for i in range(len(cell_list)//len(column_list)):
    cell_list[(6*i)].value = id_list[i]
    cell_list[(6*i)+1].value = sentence_list[i]
    cell_list[(6*i)+2].value = sentence_with_entities_list[i]
    cell_list[(6*i)+3].value = str(subject_entity_list[i])
    cell_list[(6*i)+4].value = str(object_entity_list[i])
    cell_list[(6*i)+5].value = relation_list[i]
worksheet.update_cells(cell_list)

{'spreadsheetId': '1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo',
 'updatedRange': 'annotation!A2:F2',
 'updatedRows': 1,
 'updatedColumns': 6,
 'updatedCells': 6}

#Tokenizer Pre-Trained

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
def tokenizing_(token_list, label_list, tokenizer):
  text_seq = []
  valid_seq = []
  label_seq = label_list
  for tokens, label in zip(token_list, label_list):
    new_target_text = []
    new_valid = []
    for i, (t, l) in enumerate(zip(tokens, label)):
      tokens_wordpiece = tokenizer.tokenize(t)
      new_v = [1] + [0]*(len(tokens_wordpiece)-1)
      new_target_text.extend(tokens_wordpiece)
      new_valid.extend(new_v)
    valid_seq.append(new_valid)
    text_seq.append(new_target_text)
  return text_seq, valid_seq, label_seq

In [ ]:
text_seq, valid_seq, label_seq = tokenizing_(token_list, label_list, tokenizer)

In [ ]:
print(text_seq)
print(valid_seq)
print(label_seq)

In [ ]:
def tokenizing_rel(token_list, label_list, rel_list, tokenizer):
  text_seq = []
  valid_seq = []
  final_left_seq = []
  final_right_seq = []
  label_seq = []

  for tokens, label, rel in zip(token_list, label_list, rel_list):
    lefts, rights, labels = rel

    for le, ri, la in zip(lefts, rights, labels):
      new_target_text = []
      new_valid = []
      new_left = []
      new_right = []
      before_label = -1

      for i, (t, l) in enumerate(zip(tokens, label)):
        tokens_wordpiece = tokenizer.tokenize(t)
        if before_label != l and l != -1:
          if l == le:
            tokens_wordpiece = ["*"] + tokens_wordpiece
          elif l == ri:
            tokens_wordpiece = ["#"] + tokens_wordpiece
        
        if l != -1 and i+1 != len(label) and l != label[i+1]:
          if l == le:
            tokens_wordpiece = tokens_wordpiece + ["*"]
          elif l == ri:
            tokens_wordpiece = tokens_wordpiece + ["#"]
        
        if l != -1 and i+1 == len(label):
          if l == le:
            tokens_wordpiece = tokens_wordpiece + ["*"]
          elif l == ri:
            tokens_wordpiece = tokens_wordpiece + ["#"]
        
        before_label = l
        new_v = [l]*(len(tokens_wordpiece))
        new_target_text.extend(tokens_wordpiece)
        new_valid.extend(new_v)
      
      leftaa = [float(x == le) for x in new_valid]
      rightaa = [float(x == ri) for x in new_valid]
      final_left_seq.append(leftaa)
      final_right_seq.append(rightaa)
      text_seq.append(new_target_text)
      label_seq.append(la)
    return text_seq, final_left_seq, final_right_seq, label_seq

In [ ]:
def make_greedy_label(rel_list, dic_rel_label, max_labels):
  final_rel_list = []

  for rel, max_lab in zip(rel_list, max_labels):
    temp_les = []
    temp_re =[]
    rel_labels = rel[4]
    rel_lefts = rel[5]
    rel_rights = rel[6]
    max_value = max_lab
    l, r = make_fair_by_max(max_value)
    new_l = []
    new_r = []
    qomax = len(rel_labels)
    z = 0

    for ll, rr in zip(l, r):
      q = 0
      laba = 0
      for lf, ri, rel_label in zip(rel_lefts, rel_rights, rel_labels):
        if ll == lf and rr == ri:
          q = 1
          laba = dic_rel_label[rel_label]
      
      if q == 1:
        new_l.append(ll)
        new_r.append(rr)
        temp_les.append(laba)
      elif z < 1.5*qomax:
        new_l.append(ll)
        new_r.append(rr)
        z = z + 1
        temp_les.append(0)
      
    final_rel_list.append((new_l, new_r, temp_les))
  return final_rel_list

#Relation BERT

In [16]:
import pandas as pd

In [26]:
# Dataset = pd.DataFrame(worksheet.get_all_values()).iloc[1:-1]
# column = pd.DataFrame(worksheet.get_all_values()).iloc[:1]
# Dataset.columns = column.values[0]

# print(Dataset)


      Doc          Loc       Cls       Part  \
1  sample    3704_3607   e_6_e_4  s1p5_s1p5   
2  sample    3741_3607   e_6_e_4  s1p5_s1p5   
3  sample  13057_12960   e_6_e_4  s1p6_s1p6   
4  sample  13094_12960   e_6_e_4  s1p6_s1p6   
5  sample    3394_3607  e_22_e_4  s1p5_s1p5   
6  sample    3434_3607  e_22_e_4  s1p5_s1p5   
7  sample    3455_3607  e_22_e_4  s1p5_s1p5   
8  sample  13368_13094  e_23_e_6  s1p6_s1p6   

                                                 Sub  \
1                화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것   
2  차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...   
3                화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것   
4  차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...   
5                뒷 대차 첫 번째 차축과 두 번째 차축이 진행방향 좌측으로 탈선   
6                                    좌 측 세 번째 차축이 절손   
7                    탈선화차와 14번째 화차 사이의 공기호스 가 빠져있는 것   
8  화차 차축 베어링의 분해검수 주기와 교환 주기를 화차에 적재되는 화 물의 종류, 적...   

                                                 Obj Label  
1  차축에 설치되

In [5]:
test_mlm = files.upload()

Saving MLM_1.json to MLM_1.json


In [6]:
class MLMTrainDataset_Sik(Dataset):
  
  def __init__(self, max_len, mask_prob, mask_token, sep_token, cls_token, all_sents,
               tokenizer, pad_token):
    self.all_sents = all_sents
    self.num_vocabs = tokenizer.vocab_size
    self.max_len = max_len
    self.mask_prob = mask_prob
    self.mask_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(mask_token))
    self.cls_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(cls_token))
    self.sep_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sep_token))
    self.tokenizer = tokenizer
    self.pad_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(pad_token))
  
  def __len__(self):
    return len(self.all_sents)
  
  def __getitem__(self, index):
    sents = self.all_sents[index]
    sents = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(sents))
    tokens = []
    sents = np.array(sents)
    inputs = sents
    labels = inputs.copy()

    probability_matrix = torch.full(labels.shape, self.mask_prob)

    masked_indices = torch.bernoulli(probability_matrix).bool()
    
    labels[~masked_indices] = -100

    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices

    inputs[indices_replaced] = self.tokenzier.convert_tokens_to_ids(self.mask_token)

    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced

    random_words = torch.randint(len(self.tokenizer), labels.shape, dtype = torch.long)

    inputs[indices_random] = random_words[indices_random]
    inputs = list(inputs)
    labels = list(labels)
    valid_length = min(len(inputs), self.max_len-2)
    tokens = self.cls_token + inputs[:self.max_len-2] + self.sep_token
    labels = [-100] + labels[:self.max_len-2] + [-100]

    mask_len = self.max_len - len(tokens)

    tokens = tokens + self.pad_token * mask_len
    labels = labels + [-100] * mask_len
    attention_masks = [0.0]*len(tokens)
    for i in range(valid_length):
      attention_masks[i] = 1.0
    
    return torch.LongTensor(tokens), torch.LongTensor(labels), valid_length, torch.LongTensor(attention_masks)

In [9]:
test_mlm = files.upload()

Saving sample.plain.html to sample.plain.html


In [16]:
testing = test_mlm.decode('utf-8')

AttributeError: ignored

In [8]:
sep_token="[SEP]"
pad_token="[PAD]"
cls_token="[CLS]"
mask_token="[MASK]"
max_len = 60
mask_prob = 0.15


test_instance = MLMTrainDataset_Sik(max_len, mask_prob, mask_token, sep_token, cls_token, test_mlm, BertTokenizer, pad_token)

TypeError: ignored

In [ ]:
from torchcrf import CRF

# 개체명 인식 모델 Class 코드
class Bert_entity(nn.Module):
  def __init__(self, model, num_labels):
    super().__init__()
    self.model = model
    self.hidden_size = 768
    self.block_size = 64
    self.hidden_layer = nn.Linear(768, 768)
    self.classifier = nn.Linear(768, num_labels)
    self.dropout = nn.Dropout(0.4)
    self.num_labels = num_labels
    self.crf = CRF(num_tags = num_labels, batch_first=True)
  
  def forward(self,
              input_ids = None,
              attention_mask = None,
              labels = None,
              valid_ids = None,
              label_mask = None):
    
    output = self.model(input_ids = input_ids, attention_mask = attention_mask)
    last_hidden_state = output.last_hidden_state
    valid_output = torch.zeros(last_hidden_state.size()[0], last_hidden_state.size()[1],
                               last_hidden_state.size()[2], dtype=torch.float32, device='cuda:1')
    valid_masks = torch.zeros(last_hidden_state.size()[0], last_hidden_state.size()[1],
                              dtype=torch.bool, device='cuda:1')
    
    for i in range(last_hidden_state.size()[0]):
      jj = -1
      for j in range(last_hidden_state.size()[1]):
        if valid_ids[i][j].item() == 1:
          jj += 1
          valid_masks[i][jj] = 1
          valid_output[i][jj] = last_hidden_state[i][j]
      
      relations = self.dropout(valid_output)
      logits = self.classifier(relations)
      loss = None
      if labels is not None:
        if attention_mask is not None:
          loss, logits = self.crf(logits, labels, mask = valid_masks, reduction = 'token_mean'), self.crf.decode(logits, mask=valid_masks)
        else:
          loss, logits = self.crf(logits, labels, mask = valid_masks, reduction = 'token_mean'), self.crf.decode(logits, mask=valid_masks)
      return -loss, logits, last_hidden_state, valid_output, output.attentions


In [ ]:
# 개체간 관계 추출 모델 Class 코드
class Bert_relation(nn.Module):
  def __init__(self, model, num_labels):
    super().__init__()
    self.model = model
    self.hidden_size =768
    self.block_size = 64

    self.head_extractor = nn.Linear(768, 768)
    self.tail_extractor = nn.Linear(768, 768)

    self.bilinear_classifier = nn.Linear(768 * 3, num_labels)

    self.dropout = nn.Dropout(0.1)
    self.num_labels = num_labels
  
  def get_hrts(self, valid_output, left_ids, right_ids):
    left_masks = left_ids.unsqueeze(-1).expand(valid_output.size()).float()
    right_masks = right_ids.unsqueeze(-1).expand(valid_output.size()).float()

    left_rep = torch.logsumexp(valid_output * left_masks, dim=1)
    right_rep = torch.logsumexp(valid_output * right_masks, dim=1)

    return left_rep, right_rep
  
  def forward(self,
              input_ids=None,
              attention_mask=None,
              labels=None,
              valid_ids=None,
              label_mask=None,
              left_ids=None,
              right_ids=None):
    output = self.model(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_state = output.last_hidden_state
    pooled_output = output.pooler_output
    context = self.dropout(pooled_output)
    valid_output = last_hidden_state

    left_rep, right_rep = self.get_hrts(valid_output, left_ids, right_ids)

    left_rep = self.head_extractor(left_rep)
    right_rep = self.head_extractor(right_rep)

    bl = torch.cat([context, left_rep, right_rep], dim=-1)

    logits = self.bilinear_classifier(bl)
    loss = None

    if labels is None:
      loss_fct = torch.nn.BCEWithLogitsLoss()
      loss = loss_fct(logits, labels.float())

    output = logits

    return loss, output if loss is not None else output

In [ ]:
def find_most_similar_words(word, total_words):
  q = []

  for a in total_words:
    k = jaccard_similarity(word, a[0])
    q.append((a, k))
  q.sort(key=lambda x:x[1])
  qq = q[-20:]
  new_q = []
  for qa in qq:
    if qa[1] > 0.05:
      new_q.append(qa)
  
  return new_q

In [ ]:
def jaccard_similarity(word1, word2):
  word1 = word1.lower()
  word2 = word2.lower()
  trigram1 = trigram_from_word(word1)
  trigram2 = trigram_from_word(word2)

  set_trigram = set(trigram1)
  set_trigram = set_trigram.union(set(trigram2))
  con_trigram = [t in trigram2 for t in trigram1]
  qq = sum(con_trigram)
  la = 0.0
  la = float(float(qq)/float(len(set_trigram)+1e-9))

  return la

In [ ]:
def trigram_from_word(word):
  if len(word) < 3:
    return [word]
  
  else:
    trigram = [word[i:i+3] for i in range(len(word)-3)]
    return trigram

#Sample : PDF to TXT

In [ ]:
#@title
!pip install pdfminer.six

In [ ]:
#@title
import pdfminer
from pdfminer.high_level import extract_text
from google.colab import files

pdf = files.upload()

In [ ]:
#@title
print(pdf)

In [ ]:
#@title
file_path = "sample.pdf"

text = extract_text(pdf_file=file_path)


In [ ]:
#@title
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open(file_path, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue())

#Tokenizing

In [ ]:
import re

str = output_string.getvalue()
new_str = re.sub("\n", "", str)
new_str = re.sub("·", "", new_str)

print(new_str)

In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

word_tokens = word_tokenize(new_str)
sent_tokens = sent_tokenize(new_str)

print(sent_tokens)

In [ ]:
!pip install konlpy

In [ ]:
!pip install collections

In [ ]:
from konlpy.tag import Okt
from collections import Counter
okt = Okt()
noun = okt.nouns(text)
for i, v in enumerate(noun):
  if len(v) < 2:
    noun.pop(i)

count = Counter(noun)

In [ ]:
# 단어 빈도 카운트
noun_list = count.most_common(100)
for v in noun_list:
  print(v)

#BERT Training

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

In [ ]:
inputs = bert_tokenizer(text, return_tensors='pt')
print(inputs)

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
rand = torch.rand(inputs.input_ids.shape)

mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) # 101, 102번 토큰 제외하고 15% 위치 선별

### padding이 포함된 경우(0번도 추가 제외) ###
# mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

selection = torch.flatten((mask_arr[0]).nonzero())

In [ ]:
import numpy as np

selection_val = np.random.random(len(selection)) # selection의 위치마다 0~1 값 부여

mask_selection = selection[np.where(selection_val >= 0.2)[0]] # 80% : Mask 토큰 대체
random_selection = selection[np.where(selection_val < 0.1)[0]] # 10% : 랜덤 토큰 대체

print(random_selection) # tensor([ 30,  95, 143])
print(mask_selection)

In [ ]:
inputs.input_ids[0, mask_selection] = 103
inputs.input_ids[0, random_selection] = torch.randint(0, 30522, size = random_selection.shape)

print(inputs)

In [ ]:
outputs = model(**inputs)

In [ ]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [ ]:
result = tokenizer.tokenize(new_str)
print(result)